# Twitter sentiment analyse using vaderSentiment and TextBlob
### To do this you must get your own tweeter api keys

In [30]:
# LIBRARIES:
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import csv
import os 
import string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

pd.set_option('display.max_rows', 500)

import time 
def time_elapsed(sec):
  h = int(sec/3600)
  m = int(sec/60)
  s = sec % 60
  return "{}:{:>02}:{:>05.2f}".format(h,m,s)



In [31]:
DATA_PATH = 'C:/Users/Adrian/Desktop/tweeter'
log = pd.read_csv(os.path.join(DATA_PATH, 'keys.csv')) 

# Twitter API credentials
consumerKey = log['key'][0]
consumerSecret = log['key'][1]
accesToken = log['key'][2]
accesTokenSecret =log['key'][3]

# Create autjentication object
authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret)
# Set the acces token and acces token secret
authenticate.set_access_token(accesToken,accesTokenSecret)
# Create the API object while passing in the auth information
api = tweepy.API(authenticate, wait_on_rate_limit=True)

In [32]:
# Get Tweets
HashValue = "#arsenalfc"
StartDate = "2021-01-02"
date_until = "2021-01-04"
tweets = tweepy.Cursor(api.search, q=HashValue, tweet_mode='extended', lang='en',since=StartDate,until=date_until).items(100)

In [33]:
df = pd.DataFrame([tweet.full_text for tweet in tweets], columns = ['Tweets'])
df = df.drop_duplicates()
# Cleaning the data
EMOJIS = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

def clean_text(text):
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r'@[A-Za-z0-9]+', '', text) #@mentions
    text = re.sub(r'#', '',text)
    text = re.sub(r'RT[\s]+', '',text)
    text = re.sub(r'RT[\s]+', '',text)
    for emoji in EMOJIS.keys():
        text = text.replace(emoji, '')  
    return text

df.Tweets = df.Tweets.apply(lambda x: clean_text(x))
df.head()

,Tweets
0,Fantastic SAKAMAJE 👏 Our ⭐️ boy❤️ arsenalTnu...
1,This one hurts We failed in the UEFAEuropaLeag...
2,Arsenal FC won the Premiership May they said...
3,lgergees Yes particularly noticed that Crysta...
4,❤️ Im building my own Arsenal Gunnersaurus 😍❤️...


In [34]:
# explaination about sentiments in TextBlob: 
# Each word in the lexicon has scores for:
# 1)     polarity: negative vs. positive    (-1.0 => +1.0)
# 2) subjectivity: objective vs. subjective (+0.0 => +1.0)
# https://planspace.org/20150607-textblob_sentiment
# # explaination about sentiments in Vader:
# positive >0.05
# negative <0.05
# neutral else

def get_VaderSentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    return analyzer.polarity_scores(text)['compound']
df['VaderScore'] = df['Tweets'].apply(get_VaderSentiment)
df[0:100]

def getTBSubjectivity(text):
    return TextBlob(text).sentiment[1]

def getTBPolarity(text):
    return TextBlob(text).sentiment[0]

df['TBSubjectivity'] = df['Tweets'].apply(getTBSubjectivity)
df['TBPolarity'] = df['Tweets'].apply(getTBPolarity)
#df[0:100]

In [35]:
def getVaderAnalysis(score):
    if score <= -0.05:
        return -1
    elif score >= 0.05:
        return 1
    else:
        return 0
        
df['VaderSentiment'] = df['VaderScore'].apply(getVaderAnalysis)

def getTBAnalysis(score):
    if score < 0:
        return -1
    elif score > 0:
        return 1
    else:
        return 0
        
df['TBSentiment'] = df['TBPolarity'].apply(getTBAnalysis)

df.drop(['VaderScore', 'TBSubjectivity', 'TBPolarity'], 1, inplace=True)
df[0:100]


,Tweets,VaderSentiment,TBSentiment
0,Fantastic SAKAMAJE 👏 Our ⭐️ boy❤️ arsenalTnu...,1,1
1,This one hurts We failed in the UEFAEuropaLeag...,-1,-1
2,Arsenal FC won the Premiership May they said...,1,0
3,lgergees Yes particularly noticed that Crysta...,1,1
4,❤️ Im building my own Arsenal Gunnersaurus 😍❤️...,1,1
5,Arsenal only points behind ChelseaFC when jus...,1,-1
6,I am deeeaaad FootyHumour I am laughing being ...,1,0
7,Emile Smith Rowes performance vs West Brom Sou...,1,0
8,Over followers only a small milestone since ...,1,1
9,StadiumPortrait Check out sports artwork by Pa...,0,0


In [37]:
VaderSentiment_Score = df.loc[df['VaderSentiment'] == 1, 'Tweets'].count() 
print(f'Sentiment Score = {VaderSentiment_Score*100/ len(df)}%')

VaderSentiment_nScore = df.loc[df['VaderSentiment'] == -1, 'Tweets'].count()
print(f'Sentiment NScore = {VaderSentiment_nScore*100/len(df)}%')
print(f'{VaderSentiment_Score/(VaderSentiment_nScore+VaderSentiment_Score)}%')


Sentiment Score = 75.0%
Sentiment NScore = 6.818181818181818%
0.9166666666666666%


In [38]:
df['Tweets'][96]

'Would you bring Martinelli back for the next game or play Auba given his improved performance tonight and need for a goal to boost his confidenceafc WBAARS arsenal arsenalfc PremierLeague COYG AFCRedRoom'